In [21]:
import tensorflow as tf
from xgboost import XGBRegressor as xgb
from sklearn.metrics import accuracy_score
from keras.layers import Dense
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from scipy import stats
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping

 # Load and pre-process your data
data = pd.read_csv(r"D:\DL_Homework\Kaggle2_Titanic\统计建模\数据\data-2.csv", encoding='gb18030')
data['date'] = pd.to_datetime(data['date'])
 # 添加年、月、日列
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
# print(data.head())
cols_to_convert = ['AQI指数', 'PM2.5', 'PM10', 'O3', 'no2', 'so2', 'co', 'T', 'Po', 'U', 'Ff', 'VV', 'RRR', 'year', 'month', 'day']
data[cols_to_convert] = data[cols_to_convert].astype(float)
y = data['AQI指数']
x = data.drop(['AQI指数','date'], axis=1)
 # 进行BOX-COX变换
y = stats.boxcox(y)[0]
 # 数据标准化
scaler = StandardScaler()
x = scaler.fit_transform(x)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=33)

 # Define your deep learning model
output_dim=1
model = Sequential()
model.add(Dense(10, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(1, 7)),
#     tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(output_dim, activation='softmax')
# ])

 # Train the model

 # Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
 # Train the model with early stopping
model.compile(optimizer=Adam(lr=0.01), loss='mse', metrics=['mse'])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=1000, batch_size=8, callbacks=[early_stop])
 # Extract dense features from the trained model
dense_layer_model = tf.keras.Model(inputs=model.input,
                                   outputs=model.get_layer(index=-1).output)
dense_features = dense_layer_model.predict(x_train)

# 拼接训练数据和中间层的输出
dense_features = np.concatenate((x_train, dense_features), axis=1)

 # Define XGBoost model
xgb_model = xgb()
 # Train XGBoost model on dense features
xgb_model.fit(dense_features, y_train)
 # Evaluate the XGBoost model on validation data
dense_val_features = dense_layer_model.predict(x_val)
dense_val_features = np.concatenate((x_val, dense_val_features), axis=1)
val_pred = xgb_model.predict(dense_val_features)
 # Calculate the accuracy of the model
mse = mean_squared_error(y_val, val_pred)
print("mse:", mse)

Epoch 1/1000


D:\DL_Homework\Kaggle2_Titanic\Project_Titanic\venv\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


272/272 [==============================] - 1s 3ms/step - loss: 1.1094 - mse: 1.1094 - val_loss: 0.2608 - val_mse: 0.2608
Epoch 2/1000
272/272 [==============================] - 1s 2ms/step - loss: 0.1991 - mse: 0.1991 - val_loss: 0.1456 - val_mse: 0.1456
Epoch 3/1000
272/272 [==============================] - 0s 2ms/step - loss: 0.1578 - mse: 0.1578 - val_loss: 0.1441 - val_mse: 0.1441
Epoch 4/1000
272/272 [==============================] - 1s 2ms/step - loss: 0.1488 - mse: 0.1488 - val_loss: 0.1355 - val_mse: 0.1355
Epoch 5/1000
272/272 [==============================] - 0s 2ms/step - loss: 0.1395 - mse: 0.1395 - val_loss: 0.1265 - val_mse: 0.1265
Epoch 6/1000
272/272 [==============================] - 1s 2ms/step - loss: 0.1404 - mse: 0.1404 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 7/1000
272/272 [==============================] - 1s 2ms/step - loss: 0.1377 - mse: 0.1377 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 8/1000
272/272 [==============================] - 0s 2ms/step - loss:

In [22]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))
 # 创建一个空的DataFrame
df_model_eval = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'MAE', 'R2', 'MAPE'])
 # 循环遍历所有模型，并记录评估结果

MSE = mean_squared_error(y_val, val_pred)
RMSE = np.sqrt(mean_squared_error(y_val, val_pred))
MAE = mean_absolute_error(y_val, val_pred)
R2 = r2_score(y_val, val_pred)
MAPE = mean_absolute_percentage_error(y_val, val_pred)
df_model_eval = df_model_eval.append({'Model': 'nn+xgboost',
                                          'MSE': MSE,
                                          'RMSE': RMSE,
                                          'MAE': MAE,
                                          'R2': R2,
                                          'MAPE': MAPE},
                                         ignore_index=True)
 # 输出表格
print(df_model_eval)

        Model       MSE    RMSE       MAE        R2     MAPE
0  nn+xgboost  0.115125  0.3393  0.253759  0.576116  0.05169


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12756\2005492115.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_model_eval = df_model_eval.append({'Model': 'nn+xgboost',
